In [1]:
import pandas as pd
import numpy as np
from urllib.parse import quote
import webbrowser
from tqdm.notebook import tqdm

from seleniumbase import Driver
from bs4 import BeautifulSoup
import time

from vietunits.utils import geolocator, check_point_in_polygon, find_nearest_point

import warnings
warnings.filterwarnings('ignore')

In [2]:
df = pd.read_csv('data/danhmuc_and_sapnhap.csv')

In [3]:
df.shape

(10602, 19)

In [4]:
# Các ward bị chia
df_divided = df[df['isDividedWard']==True].copy()
df_divided.sort_values(by=['province', 'district', 'ward', 'newWardAreaKm2'], inplace=True)

In [5]:
df_divided

,provinceCode,isMergedProvince,districtCode,districtType,wardCode,wardType,isMergedWard,isDividedWard,province,district,ward,newProvince,newWard,newProvinceCode,newWardCode,newWardType,newWardLat,newWardLon,newWardAreaKm2
10208,92,True,927,Huyện,31268.0,Xã,True,True,Thành phố Cần Thơ,Huyện Thới Lai,Xã Tân Thạnh,Thành phố Cần Thơ,Trường Thành,33,3275.0,xã,10.04230,105.610,59.09
10207,92,True,927,Huyện,31268.0,Xã,True,True,Thành phố Cần Thơ,Huyện Thới Lai,Xã Tân Thạnh,Thành phố Cần Thơ,Tân Thạnh,33,3254.0,xã,9.62301,106.064,70.80
10185,92,True,924,Huyện,31246.0,Xã,True,True,Thành phố Cần Thơ,Huyện Vĩnh Thạnh,Xã Thạnh Quới,Thành phố Cần Thơ,Gia Hòa,33,3204.0,xã,9.43175,105.800,77.06
10186,92,True,924,Huyện,31246.0,Xã,True,True,Thành phố Cần Thơ,Huyện Vĩnh Thạnh,Xã Thạnh Quới,Thành phố Cần Thơ,Thạnh Quới,33,3258.0,xã,10.20750,105.348,103.86
10155,92,True,918,Quận,31178.0,Phường,True,True,Thành phố Cần Thơ,Quận Bình Thủy,Phường Bùi Hữu Nghĩa,Thành phố Cần Thơ,Cái Khế,33,3193.0,phường,10.05170,105.781,10.04
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9791,87,True,874,Huyện,30157.0,Xã,True,True,Tỉnh Đồng Tháp,Huyện Thanh Bình,Xã Tân Thạnh,Tỉnh Đồng Tháp,Thanh Bình,31,3069.0,xã,10.41230,106.386,86.00
9699,87,True,867,Thành phố,29902.0,Phường,True,True,Tỉnh Đồng Tháp,Thành phố Sa Đéc,Phường 3,Tỉnh Đồng Tháp,Sa Đéc,31,3046.0,phường,10.32900,105.739,46.90
9700,87,True,867,Thành phố,29902.0,Phường,True,True,Tỉnh Đồng Tháp,Thành phố Sa Đéc,Phường 3,Tỉnh Đồng Tháp,Cao Lãnh,31,2997.0,phường,10.43990,105.624,73.30
9702,87,True,867,Thành phố,29908.0,Phường,True,True,Tỉnh Đồng Tháp,Thành phố Sa Đéc,Phường 4,Tỉnh Đồng Tháp,Sa Đéc,31,3046.0,phường,10.32900,105.739,46.90


# Default newWard when street is not provided

In [6]:
df_divided_ward = df_divided[['province', 'district', 'ward']].drop_duplicates()

In [7]:
# for index, row in tqdm(df_divided_ward.iterrows(), total=len(df_divided_ward)):
#     address = row['ward'] + ', ' + row['district'] + ', ' + row['province']
#     location = geolocator.geocode(address)
#     df_divided_ward.loc[index, ['wardLat', 'wardLon', 'wardGeoAddress']] = location.latitude, location.longitude, location.address
# df_divided_ward.to_csv('data/df_divided_ward.csv', index=False)
# df_divided_ward = pd.read_csv('data/df_divided_ward.csv')

In [8]:
driver = Driver(uc=True)

In [14]:
driver.get('https://developers-dot-devsite-v2-prod.appspot.com/maps/documentation/utils/geocoder?hl=vi')

In [15]:
def get_location(address):
    driver.send_keys('#query-input', address)
    driver.click('#geocode-button')
    driver.find_element('#query-input').clear()
    time.sleep(10)
    html = driver.get_page_source()
    soup = BeautifulSoup(html, 'html.parser')
    results_display = soup.find(id='results-display-div')
    result_location = results_display.find(class_='result-location')
    location = result_location.text.split()[1]
    try:
        result_bounds = results_display.find(class_='result-bounds')
        bounds = ' '.join(result_bounds.text.split()[1:])
    except:
        try:
            result_viewport = results_display.find(class_='result-viewport')
            bounds = ' '.join(result_viewport.text.split()[1:])
        except:
            bounds = None

    try:
        result_formatted_address = results_display.find(class_='result-formatted-address')
        formatted_address = result_formatted_address.text.split('\n')[-1].strip()
    except:
        formatted_address = None

    data = {
        'wardLat': location.split(',')[0],
        'wardLon': location.split(',')[1],
        'wardBounds': bounds,
        'wardFormattedAddress': formatted_address,
    }

    return data

In [12]:
df_divided_ward[df_divided_ward['wardLat'].isna()]

,province,district,ward,wardLat,wardLon,wardBounds,wardFormattedAddress
3895,Thành phố Hải Phòng,Huyện Kiến Thụy,Xã Kiến Hưng,NaN,NaN,NaN,NaN
3899,Thành phố Hải Phòng,Huyện Kiến Thụy,Xã Đoàn Xá,NaN,NaN,NaN,NaN
3912,Thành phố Hải Phòng,Huyện Tiên Lãng,Xã Tân Minh,NaN,NaN,NaN,NaN
3940,Thành phố Hải Phòng,Huyện Vĩnh Bảo,Xã Hòa Bình,NaN,NaN,NaN,NaN
3860,Thành phố Hải Phòng,Quận An Dương,Phường An Hải,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...
9763,Tỉnh Đồng Tháp,Huyện Cao Lãnh,Xã Gáo Giồng,NaN,NaN,NaN,NaN
9734,Tỉnh Đồng Tháp,Huyện Hồng Ngự,Xã Phú Thuận B,NaN,NaN,NaN,NaN
9790,Tỉnh Đồng Tháp,Huyện Thanh Bình,Xã Tân Thạnh,NaN,NaN,NaN,NaN
9699,Tỉnh Đồng Tháp,Thành phố Sa Đéc,Phường 3,NaN,NaN,NaN,NaN


In [ ]:
for index, row in tqdm(df_divided_ward[df_divided_ward['wardLat'].isna()].iterrows(), total=len(df_divided_ward[df_divided_ward['wardLat'].isna()])):
    address = row['ward'] + ', ' + row['district'] + ', ' + row['province']
    location = get_location(address)
    df_divided_ward.loc[index, ['wardLat', 'wardLon', 'wardBounds', 'wardFormattedAddress']] = location['wardLat'], location['wardLon'], location['wardBounds'], location['wardFormattedAddress']
    print(location)
df_divided_ward.to_csv('data/df_divided_ward.csv', index=False)
# df_divided_ward = pd.read_csv('data/df_divided_ward.csv')

  0%|          | 0/265 [00:00<?, ?it/s]

In [ ]:
df_divided_ward

In [ ]:
# for index, row in df_divided_ward.iterrows():
#     province = row['province']
#     district = row['district']
#     ward = row['ward']
#     ward_point = (row['wardLat'], row['wardLon'])
#
#     new_wards = df[(df['province']==province) & (df['district']==district) & (df['ward']==ward)]
#
#     containing_points= []
#     new_ward_points = []
#     for new_ward_index, new_ward_row in new_wards.iterrows():
#         new_ward_point = (new_ward_row['newWardLat'], new_ward_row['newWardLon'])
#         new_ward_area_km2 = new_ward_row['newWardAreaKm2']
#         new_ward_points.append(new_ward_point)
#
#         is_contain = check_point_in_polygon(point=ward_point, polygon_center=new_ward_point, polygon_area_km2=new_ward_area_km2)
#         if is_contain:
#             containing_points.append(new_ward_point)
#         df.loc[new_ward_index, 'isNewWardPolygonContainsWard'] = is_contain
#
#     nearest_point = find_nearest_point(a_point=ward_point, list_of_b_points=new_ward_points)
#
#     if len(containing_points) == 1:
#         default_ward_point = containing_points[0]
#     else:
#         default_ward_point = nearest_point
#
#
#     df.loc[(df['province']==province) & (df['district']==district) & (df['ward']==ward) & (df['newWardLat']==nearest_point[0])& (df['newWardLon']==nearest_point[1]), 'isNearestNewWard'] = True
#     df.loc[(df['province']==province) & (df['district']==district) & (df['ward']==ward) & (df['newWardLat']==default_ward_point[0])& (df['newWardLon']==default_ward_point[1]), 'isDefaultNewWard'] = True
#
# df.loc[(df['isDividedWard']==True) & (df['isNearestNewWard'].isna()), 'isNearestNewWard'] = False
# df.loc[(df['isDividedWard']==True) & (df['isDefaultNewWard'].isna()), 'isDefaultNewWard'] = False

In [ ]:
# df.to_csv('data/danhmuc_and_sapnhap_has_default_new_ward.csv', index=False)

# Allocate streets to new ward

In [ ]:
# Các district có ward bị chia
df_divided_district = df_divided[['province', 'district']].drop_duplicates()
#
# # # Tìm tên đường phố thủ công
# # for index, row in tqdm(df_divided_district.iterrows(), total=df_divided_district.shape[0]):
# #     keyword = row['district'] + ', ' + row['province'] + ' wikipedia'
# #     search_url = f"https://www.google.com/search?q={quote(keyword)}"
# #     webbrowser.open(search_url)
# #     wikipedia = input("wikipedia:").upper()
# #     if wikipedia != 'NO':
# #         element = input("element:")
# #     else:
# #         element = 'NO'
# #     df_divided_district.loc[index, 'wikipedia'] = wikipedia
# #     df_divided_district.loc[index, 'element'] = element
# # df_divided_district.to_csv('data/df_divided_district.csv', index=False)
#
df_divided_district_2 = pd.read_csv('data/df_divided_district.csv')

In [ ]:
df_divided_district

In [ ]:
df_divided_district_2

In [ ]:
pd.merge(df_divided_district, df_divided_district_2, on=['province', 'district'], how='left')

In [ ]:
import requests
import json

# Overpass API endpoint
url = "https://overpass-api.de/api/interpreter"

# Overpass QL query: Lấy tên đường trong Phường 15, Quận Tân Bình, Hồ Chí Minh


# Gửi request
response = requests.post(url, data={'data': query})

# Kiểm tra kết quả
if response.status_code == 200:
    data = response.json()
    # Lọc danh sách tên đường duy nhất
    street_names = set()
    for element in data['elements']:
        if element['type'] == 'way' and 'tags' in element and 'name' in element['tags']:
            street_names.add(element['tags']['name'])

    # In ra danh sách tên đường
    print("Danh sách tên đường trong Phường 15, Quận Tân Bình, Hồ Chí Minh:")
    for name in sorted(street_names):
        print("-", name)
else:
    print("Lỗi khi truy vấn:", response.status_code)

In [ ]:
def get_streets(province: str, district: str, ward: str) -> list:
    url = "https://overpass-api.de/api/interpreter"
    query = f"""
    [out:json][timeout:25];

    // Bước 1: Xác định TP.HCM
    area["name"="{province}"]["boundary"="administrative"]["admin_level"="4"]->.tp;

    // Bước 2: Trong TP.HCM, tìm Quận Tân Bình
    area(area.tp)["name"="{district}"]["boundary"="administrative"]["admin_level"="6"]->.quan;

    // Bước 3: Trong Quận Tân Bình, tìm Phường 15
    area(area.quan)["name"="{ward}"]["boundary"="administrative"]["admin_level"="8"]->.phuong;

    // Bước 4: Lấy các đường trong phường đó
    way(area.phuong)["highway"]["name"];
    out body;
    >;
    out skel qt;
    """

    # Gửi request
    response = requests.post(url, data={'data': query})

    # Kiểm tra kết quả
    if response.status_code == 200:
        data = response.json()
        # Lọc danh sách tên đường duy nhất
        street_names = set()
        for element in data['elements']:
            if element['type'] == 'way' and 'tags' in element and 'name' in element['tags']:
                street_names.add(element['tags']['name'])

        # street_names = (clean_street_name(name) for name in street_names)

        return list(street_names)

    else:
        return []

In [ ]:
streets = get_streets(province='Thành phố Hồ Chí Minh', district='quận Tân Bình', ward='Phường 15')

In [ ]:
streets

In [ ]:
import re
def extract_street_name(text):
    text =  re.sub(r'^Hẻm\s?[\w\/\-]*\s', '', text, flags=re.IGNORECASE).strip()
    if re.search('Đường', text, flags=re.IGNORECASE) and not re.search('Đường số', text, flags=re.IGNORECASE):
        text =  re.sub('Đường ', '', text, flags=re.IGNORECASE).strip()
    text = text.title()
    return text

In [ ]:
def clean_street_name(text):
    text = text.lower()
    if text.startswith('đường'):
        texts = text.split()
        second_text = texts[1]
        if not re.search('\d', second_text, flags=re.IGNORECASE) and 'số' not in second_text:
            text = ' '.join(texts[1:])
    return text.title()

In [ ]:
extract_street_name('130 Mễ Cốc')

In [ ]:
streets

In [ ]:
set([clean_street_name(s) for s in streets if 'Hẻm' not in s])

In [ ]:
clean_street_name('Đường Bình Đức')

e# Không có street, default

In [ ]:
geolocator.geocode('Phường 15, Quận Tân Bình, Hồ Chí Minh')

In [ ]:
address = 'Phường 15, Quận Tân Bình, Hồ Chí Minh'

In [ ]:
driver.send_keys('#query-input', address)
driver.click('#geocode-button')
driver.find_element('#query-input').clear()
time.sleep(3)
html = driver.get_page_source()
soup = BeautifulSoup(html, 'html.parser')
results_display = soup.find(id='results-display-div')
result_location = results_display.find(class_='result-location')
location = result_location.text.split()[1]
result_bounds = results_display.find(class_='result-bounds')
bounds = ' '.join(result_bounds.text.split()[1:])

In [ ]:
driver.quit()

In [ ]:
def get_location(address):
    driver.send_keys('#query-input', address)
    driver.click('#geocode-button')
    driver.find_element('#query-input').clear()
    time.sleep(3)
    html = driver.get_page_source()
    soup = BeautifulSoup(html, 'html.parser')
    results_display = soup.find(id='results-display-div')
    result_location = results_display.find(class_='result-location')
    location = result_location.text.split()[1]
    result_bounds = results_display.find(class_='result-bounds')
    bounds = ' '.join(result_bounds.text.split()[1:])

    data = {
        'wardAddress': address,
        'wardLat': location.split(',')[0],
        'wardLon': location.split(',')[1],
        'wardBounds': bounds
    }

    return data